## Project for Coursera Capstone

## Introduction

In this project we will explore if someone wants to open a business at Manhattan Area of New York City what will be the recommended neighborhood. 

## Business Problem

The Manhattan area of New York is the economic and administrative center. It is often described as the cultural, financial, media, and entertainment capital of the world. Thus it offers a great opportunity for someone who is trying to open a new shop. However being a diverse metropolitan it also pose certain challanges to figure out where to start a new business.

It would be nice if there is a recommender system which analyze the different neighborhoods and recommend the best area to open the business. This machine learning recommender system will help the small business owner make a informed decision on best areas to open the business.


## Data

The recommender system will use the Foursquare location data to get details of the presense of similar type of business in each neighborhood. Foursquare rest-based API provide details like Venue Name, Catagory, customer likes, etc. which are very useful datapoints to make the decision. 

It will then try to use the clustering algorithm to group the neighborhoods and provide addition details of those neighbourhood. 

The data from Foursquare will be fetched using the longitude and latiatude details of neighborhoods around Manhattan. 

## Methodology 

### i. Environment Setup

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from tqdm import tqdm

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  49.48 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.45 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  41.02 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  45.75 MB/s
Libraries imported.


### ii. Import Data

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


### iii. Load and explore the data

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

All the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [4]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [5]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

#### Tranform the data into a *pandas* dataframe

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
# let's loop through the data and fill the dataframe one row at a time.

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [9]:
# Filter the neighborhoods for Manhattan only

manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [10]:
print('The Manhattan borough has {} neighborhoods.'.format(
        manhattan_data.shape[0]
    )
)

The Manhattan borough has 40 neighborhoods.


### iv. Map the Manhattan neighborhoods

#### Let's get the geographical coordinates of Manhattan.

In [11]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


Let's visualize Manhattan the neighborhoods on the map

In [12]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### v. Get the venues around Manhattan using Foresquare

#### Define Foursquare Credentials and Version

In [13]:
# The code was removed by Watson Studio for sharing.

#### Function that extracts the category of the venue from all the neighborhoods in Manhattan

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=200):
    
    pbar = tqdm(total=36)
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):        
        pbar.update(1)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        
    pbar.close()
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Fetch all the venues around Manhattan 

In [15]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

print('Manhattan has {} Neighborhood and total {} venues.'.format(
        len(manhattan_venues['Neighborhood'].unique()),
        manhattan_venues.shape[0]
    )
)

40it [00:25,  1.52it/s]                        

Manhattan has 40 Neighborhood and total 3298 venues.


In [16]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


#### Let's find out how many unique categories can be curated from all the returned venues

In [17]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 330 uniques categories.


### vi. Analyze Each Neighborhood

In [18]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South In

In [19]:
manhattan_onehot.shape

(3298, 331)

#### Filter the Venues which are type Resturant

In [20]:
manhattan_restaurant = manhattan_onehot.filter(regex='Neighborhood|Restaurant|Pizza|Coffee|Café|Food Court')

manhattan_restaurant.shape

(3298, 81)

#### Filter the Venues which are type other than Resturant

In [21]:
other_cols = [col for col in manhattan_onehot.columns if 'Restaurant' not in col\
                                                     and 'Pizza' not in col\
                                                     and 'Coffee' not in col\
                                                     and 'Café' not in col\
                                                     and 'Food Court' not in col]

manhattan_others = manhattan_onehot[other_cols]
manhattan_others.shape

(3298, 251)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of restaurant each category

In [22]:
manhattan_restaurant_grouped = manhattan_restaurant.groupby('Neighborhood').mean().reset_index()
manhattan_restaurant_grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Belgian Restaurant,Brazilian Restaurant,Café,Cambodian Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Coffee Shop,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food Court,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,Battery Park City,0.0,0.000000,0.010000,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.010000,0.08,0.0,0.0,0.00,0.00,0.0,0.00,0.000000,0.0,0.0,0.00,0.01,0.000000,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.03,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.01,0.01,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.020000,0.0,0.00,0.00,0.0,0.0,0.010000,0.00,0.0,0.0,0.000000,0.00,0.01,0.0,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.00,0.0,0.00
1,Carnegie Hill,0.0,0.000000,0.010000,0.0,0.01,0.00,0.0,0.00,0.0,0.0,0.040000,0.0,0.000000,0.0,0.000000,0.05,0.0,0.0,0.00,0.00,0.0,0.00,0.000000,0.0,0.0,0.01,0.00,0.030000,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.02,0.0,0.03,0.0,0.0,0.01,0.0,0.0,0.00,0.01,0.02,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.0,0.0,0.0,0.060000,0.0,0.01,0.01,0.0,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.01,0.0,0.0,0.00,0.000000,0.01,0.0,0.01,0.0,0.01,0.0,0.02
2,Central Harlem,0.0,0.066667,0.044444,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.022222,0.0,0.022222,0.0,0.044444,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.022222,0.0,0.0,0.00,0.00,0.044444,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.022222,0.0,0.00,0.00,0.0,0.0,0.044444,0.00,0.0,0.0,0.022222,0.00,0.00,0.0,0.0,0.00,0.022222,0.00,0.0,0.00,0.0,0.00,0.0,0.00
3,Chelsea,0.0,0.000000,0.030000,0.0,0.00,0.02,0.0,0.00,0.0,0.0,0.010000,0.0,0.000000,0.0,0.000000,0.07,0.0,0.0,0.00,0.00,0.0,0.00,0.000000,0.0,0.0,0.00,0.00,0.010000,0.0,0.0,0.00,0.0,0.00,0.01,0.01,0.06,0.0,0.01,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.01,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.010000,0.0,0.00,0.00,0.0,0.0,0.030000,0.00,0.0,0.0,0.000000,0.00,0.01,0.0,0.0,0.00,0.020000,0.00,0.0,0.00,0.0,0.01,0.0,0.00
4,Chinatown,0.0,0.000000,0.040000,0.0,0.00,0.02,0.0,0.01,0.0,0.0,0.000000,0.0,0.000000,0.0,0.090000,0.02,0.0,0.0,0.04,0.02,0.0,0.01,0.000000,0.0,0.0,0.00,0.00,0.000000,0.0,0.0,0.01,0.0,0.03,0.00,0.00,0.01,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.02,0.00,0.01,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.0,0.0,0.0,0.010000,0.0,0.01,0.00,0.0,0.0,0.000000,0.01,0.0,0.0,0.000000,0.01,0.00,0.0,0.0,0.01,0.000000,0.00,0.0,0.00,0.0,0.00,0.0,0.04


Let's confirm the new size

In [23]:
manhattan_restaurant_grouped.shape

(40, 81)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of other venues

In [24]:
manhattan_others_grouped = manhattan_others.groupby('Neighborhood').mean().reset_index()
manhattan_others_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Animal Shelter,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Athletics & Sports,Auditorium,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Camera Store,Candy Store,Cheese Shop,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Duty-free Shop,Electronics Store,Event Space,Exhibit,Farmers Market,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Fountain,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Indie Theater,Irish Pub,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Memorial Site,Men's Store,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Social Club,Soup Place,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Taco Place,Tailor Shop,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Trail,Tree,Used Bookstore,Veterinarian,Video Game Store,Video Store,Volleyball Court,Watch Shop,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.01,0.01,0.0,0.020000,0.0,0.000000,0.01,0.0,0.000000,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.00,0.0,0.01,0.0,0.02,0.000000,0.000000,0.0,0.00,0.0,0.00,0.01,0.02,0.01,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.01,0.0,0.0,0.0,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.010000,0.00,0.00,0.0,0.02,0.000000,0.00,0.0,0.0,0.02,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.000000,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.02,0.02,0.000000,0.0,0.00,0.0,0.01,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.01,0.01,0.030000,0.010000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.00,0.05,0.01,0.02,0.00,0.000000,0.0,0.000000,0.00,0.010000,0.00,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0

Let's confirm the new size

In [25]:
manhattan_restaurant_grouped.shape

(40, 81)

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 Restaurant for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Restaurant'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Restaurant'.format(ind+1))

# create a new dataframe
neighborhoods_restaurant_sorted = pd.DataFrame(columns=columns)
neighborhoods_restaurant_sorted['Neighborhood'] = manhattan_restaurant_grouped['Neighborhood']

for ind in np.arange(manhattan_restaurant_grouped.shape[0]):
    neighborhoods_restaurant_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_restaurant_grouped.iloc[ind, :], num_top_venues)

neighborhoods_restaurant_sorted.head()

,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
0,Battery Park City,Coffee Shop,Italian Restaurant,Pizza Place,American Restaurant,Chinese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Sushi Restaurant,Seafood Restaurant,Food Court
1,Carnegie Hill,Pizza Place,Coffee Shop,Café,Japanese Restaurant,French Restaurant,Italian Restaurant,Mexican Restaurant,Vietnamese Restaurant,Argentinian Restaurant,Mediterranean Restaurant
2,Central Harlem,African Restaurant,American Restaurant,Chinese Restaurant,Seafood Restaurant,French Restaurant,Café,Southern / Soul Food Restaurant,Pizza Place,Caribbean Restaurant,Ethiopian Restaurant
3,Chelsea,Coffee Shop,Italian Restaurant,American Restaurant,Seafood Restaurant,Tapas Restaurant,Asian Restaurant,Paella Restaurant,Pizza Place,Israeli Restaurant,Mexican Restaurant
4,Chinatown,Chinese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,American Restaurant,Hotpot Restaurant,Coffee Shop,Malay Restaurant,Dumpling Restaurant,Asian Restaurant,Taiwanese Restaurant


#### Now let's create the new dataframe and display the top 10 Venues for each neighborhood other than Restaurant.

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_others_sorted = pd.DataFrame(columns=columns)
neighborhoods_others_sorted['Neighborhood'] = manhattan_others_grouped['Neighborhood']

for ind in np.arange(manhattan_others_grouped.shape[0]):
    neighborhoods_others_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_others_grouped.iloc[ind, :], num_top_venues)

neighborhoods_others_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Wine Shop,Gym,Women's Store,Fountain,Shopping Mall,Ice Cream Shop,Food Truck,Clothing Store
1,Carnegie Hill,Cosmetics Shop,Yoga Studio,Spa,Bar,Gym,Bookstore,Wine Shop,Pub,Gym / Fitness Center,Sports Bar
2,Central Harlem,Fried Chicken Joint,Cosmetics Shop,Gym / Fitness Center,History Museum,Library,Beer Bar,Market,Bookstore,Boutique,Music Venue
3,Chelsea,Ice Cream Shop,Bakery,Nightclub,Hotel,Theater,Art Gallery,Bookstore,Gift Shop,Market,Health & Beauty Service
4,Chinatown,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop,Salon / Barbershop,Noodle House,Bakery,Furniture / Home Store,Optical Shop,Sandwich Place,Spa


### vi. Cluster Neighborhoods by Venues - Type other than Restaurants

Run *k*-means to cluster the neighborhood into 5 clusters.

In [29]:
# set number of clusters
kclusters = 5

manhattan_others_clustering = manhattan_others_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_others_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 2, 0, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [30]:
manhattan_others_merged = manhattan_data

# add clustering labels
manhattan_others_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_others_merged = manhattan_others_merged.join(neighborhoods_others_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_others_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Discount Store,Yoga Studio,Kids Store,Deli / Bodega,Pharmacy,Department Store,Diner,Sandwich Place,Donut Shop,Shoe Store
1,Manhattan,Chinatown,40.715618,-73.994279,1,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop,Salon / Barbershop,Noodle House,Bakery,Furniture / Home Store,Optical Shop,Sandwich Place,Spa
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Bakery,Mobile Phone Shop,Sandwich Place,Grocery Store,Donut Shop,Women's Store,Shoe Store,Deli / Bodega,Supermarket,Gym
3,Manhattan,Inwood,40.867684,-73.921210,1,Lounge,Bakery,Deli / Bodega,Wine Bar,Park,Frozen Yogurt Shop,Yoga Studio,Bus Station,Laundromat,Supermarket
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Deli / Bodega,Yoga Studio,School,Sandwich Place,Cocktail Bar,Bakery,Liquor Store,Park,Donut Shop,Bar


#### Map Restaurant Clustering 

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_others_merged['Latitude'],\
                                  manhattan_others_merged['Longitude'],\
                                  manhattan_others_merged['Neighborhood'],\
                                  manhattan_others_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### vii. Examine the Venue Clusters

#### Cluster 1

In [32]:
manhattan_others_merged.loc[manhattan_others_merged['Cluster Labels'] == 0,\
                                manhattan_others_merged.columns[[1]\
                              + list(range(5, manhattan_others_merged.shape[1] -5))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,East Harlem,Bakery,Deli / Bodega,Steakhouse,Cocktail Bar,Street Art
13,Lincoln Square,Theater,Gym / Fitness Center,Plaza,Concert Hall,Park
15,Midtown,Hotel,Steakhouse,Theater,Food Truck,Bakery
20,Lower East Side,Sandwich Place,Shoe Store,Art Gallery,Bakery,Cocktail Bar
21,Tribeca,Spa,Park,Gym,Wine Bar,Boutique
22,Little Italy,Bakery,Yoga Studio,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop
25,Manhattan Valley,Bar,Deli / Bodega,Playground,Yoga Studio,Juice Bar
33,Midtown South,Hotel Bar,Cosmetics Shop,Bakery,Boutique,Cocktail Bar
37,Stuyvesant Town,Bar,Boat or Ferry,Park,Farmers Market,Basketball Court


#### Cluster 2

In [33]:
manhattan_others_merged.loc[manhattan_others_merged['Cluster Labels'] == 1,\
                                manhattan_others_merged.columns[[1]\
                              + list(range(5, manhattan_others_merged.shape[1] -5))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Marble Hill,Discount Store,Yoga Studio,Kids Store,Deli / Bodega,Pharmacy
1,Chinatown,Cocktail Bar,Bubble Tea Shop,Ice Cream Shop,Salon / Barbershop,Noodle House
2,Washington Heights,Bakery,Mobile Phone Shop,Sandwich Place,Grocery Store,Donut Shop
3,Inwood,Lounge,Bakery,Deli / Bodega,Wine Bar,Park
4,Hamilton Heights,Deli / Bodega,Yoga Studio,School,Sandwich Place,Cocktail Bar
5,Manhattanville,Deli / Bodega,Other Nightlife,Park,Music School,Burger Joint
8,Upper East Side,Exhibit,Art Gallery,Bakery,Hotel,Gym / Fitness Center
9,Yorkville,Bar,Gym,Deli / Bodega,Pub,Spa
10,Lenox Hill,Gym / Fitness Center,Sporting Goods Shop,Burger Joint,Gym,Cycle Studio
11,Roosevelt Island,Sandwich Place,Park,Indie Theater,School,Liquor Store


#### Cluster 3

In [34]:
manhattan_others_merged.loc[manhattan_others_merged['Cluster Labels'] == 2,\
                                manhattan_others_merged.columns[[1]\
                              + list(range(5, manhattan_others_merged.shape[1] -5))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,Central Harlem,Fried Chicken Joint,Cosmetics Shop,Gym / Fitness Center,History Museum,Library
14,Clinton,Theater,Gym / Fitness Center,Wine Shop,Gym,Spa
17,Chelsea,Ice Cream Shop,Bakery,Nightclub,Hotel,Theater


#### Cluster 4

In [35]:
manhattan_others_merged.loc[manhattan_others_merged['Cluster Labels'] == 3,\
                                manhattan_others_merged.columns[[1]\
                              + list(range(5, manhattan_others_merged.shape[1] -5))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
30,Carnegie Hill,Cosmetics Shop,Yoga Studio,Spa,Bar,Gym


#### Cluster 5

In [36]:
manhattan_others_merged.loc[manhattan_others_merged['Cluster Labels'] == 4,\
                                manhattan_others_merged.columns[[1]\
                              + list(range(5, manhattan_others_merged.shape[1] -5))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
28,Battery Park City,Park,Hotel,Wine Shop,Gym,Women's Store


### viii. Analyze which neighborhood is best for opening a new Restaurant

In [37]:
restaurant_type = input("Enter the restaurant type.. ")

Enter the restaurant type.. Indian


In [38]:
restaurant_type

'Indian'

#### List the neighborhood which has the specific resturant in top 10 common venues

In [39]:
col_num = neighborhoods_restaurant_sorted.shape[1]

# define the dataframe columns
col_names = ['Neighborhood', 'Type', 'Most Common'] 

# instantiate the dataframe
is_common_restaurant = pd.DataFrame(columns=col_names)

for index, row in neighborhoods_restaurant_sorted.iterrows():
    for i in range (1, col_num):
        if restaurant_type in row[i]:
            is_common_restaurant = is_common_restaurant.append({'Neighborhood': row[0],
                                          'Type': row[i], 'Most Common': i}, ignore_index=True)
is_common_restaurant.shape

(11, 3)

In [40]:
manhattan_restaurant_filtered = is_common_restaurant

manhattan_restaurant_filtered = manhattan_restaurant_filtered.join(manhattan_data.set_index('Neighborhood'), on='Neighborhood')

manhattan_restaurant_filtered.drop(['Borough'], axis=1, inplace=True)
manhattan_restaurant_filtered

,Neighborhood,Type,Most Common,Latitude,Longitude,Cluster Labels
0,Greenwich Village,Indian Restaurant,7,40.726933,-73.999914,1
1,Hamilton Heights,Indian Restaurant,8,40.823604,-73.949688,1
2,Lenox Hill,Indian Restaurant,9,40.768113,-73.958860,1
3,Manhattan Valley,Indian Restaurant,4,40.797307,-73.964286,0
4,Manhattanville,Indian Restaurant,5,40.816934,-73.957385,1
5,Midtown,Indian Restaurant,8,40.754691,-73.981669,0
6,Morningside Heights,Indian Restaurant,5,40.808000,-73.963896,1
7,Sutton Place,Indian Restaurant,2,40.760280,-73.963556,1
8,Turtle Bay,Indian Restaurant,4,40.752042,-73.967708,1
9,Upper West Side,Indian Restaurant,4,40.787658,-73.977059,1


### Map the Neighborhood where specific restaurant is common

In [41]:
# create map
map_resturant = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
rainbow = colors = ['red', 'blue', 'gray', 'darkred', 'lightred', 'orange', 'beige',
    'green', 'darkgreen', 'lightgreen', 'darkblue', 'lightblue', 'purple', 'darkpurple',
    'pink', 'cadetblue', 'lightgray', 'black']

# add markers to the map
markers_colors = []
for lat, lon, poi, common in zip(manhattan_restaurant_filtered['Latitude'],\
                                  manhattan_restaurant_filtered['Longitude'],\
                                  manhattan_restaurant_filtered['Neighborhood'],\
                                  manhattan_restaurant_filtered['Most Common']):
    label = folium.Popup(str(poi) + ' Most Common: ' + str(common), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[common-1],
        fill=True,
        fill_color=rainbow[common-1],
        fill_opacity=0.7).add_to(map_resturant)
       
map_resturant

### Find neighborhood where the specific restaurant is not in top 10

In [75]:

manhattan_restaurant_suitable = manhattan_data

manhattan_restaurant_suitable = manhattan_restaurant_suitable.join(is_common_restaurant.set_index('Neighborhood'), on='Neighborhood')

manhattan_restaurant_suitable.drop(['Borough'], axis=1, inplace=True)

manhattan_restaurant_neighborhood_recommendation = manhattan_restaurant_suitable.loc[manhattan_restaurant_suitable['Most Common'].isnull()]
manhattan_restaurant_neighborhood_recommendation.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,Type,Most Common
0,Marble Hill,40.876551,-73.910660,1,NaN,NaN
1,Chinatown,40.715618,-73.994279,1,NaN,NaN
3,Inwood,40.867684,-73.921210,1,NaN,NaN
6,Central Harlem,40.815976,-73.943211,2,NaN,NaN
7,East Harlem,40.792249,-73.944182,0,NaN,NaN


## Results 

### Map the neighborhood 

In [81]:
# create map
map_resturant = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
rainbow = colors = ['darkgreen','darkblue','purple', 'darkpurple',
    'pink', 'cadetblue', 'lightgray', 'black', 'red', 'blue', 'gray', 'darkred', 'lightred', 'orange', 'beige']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_restaurant_neighborhood_recommendation['Latitude'],\
                                  manhattan_restaurant_neighborhood_recommendation['Longitude'],\
                                  manhattan_restaurant_neighborhood_recommendation['Neighborhood'],\
                                  manhattan_restaurant_neighborhood_recommendation['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_resturant)
       
map_resturant

### The above neighborhoods are some of the suggested areas to open the restaurant

## Conclusion

Thus using the location data from Foresquare along with Machine Learning algorithm we can use a recommender system that will help to guide small business owners to make informative decision on which is the best neighborhood to a new restaurant.